In [0]:
import numpy as np
from google.colab import files
import pandas as pd
uploaded = files.upload()

Saving Housing.csv to Housing (1).csv


In [0]:
import io
data = pd.read_csv(io.BytesIO(uploaded['Housing.csv']), delim_whitespace=True)

In [0]:
from keras.utils import np_utils

In [0]:
y = data.drop(data.columns[-1], axis=1)
x = data[data.columns[-1]]
#y = data.drop('variety', axis=1)
#x = data.variety.astype('category').cat.codes
#y = y.rename(columns=y.iloc[0]).drop(y.index[0])
y = y.to_numpy()
x = x.to_numpy()
#x = np.delete(x, 0)
y = y.reshape(505, 1, 13)
x = x.reshape(505, 1, 1)

In [0]:
x = x / x.max()
x

array([[[0.432]],

       [[0.694]],

       [[0.668]],

       [[0.724]],

       [[0.574]],

       [[0.458]],

       [[0.542]],

       [[0.33 ]],

       [[0.378]],

       [[0.3  ]],

       [[0.378]],

       [[0.434]],

       [[0.408]],

       [[0.364]],

       [[0.398]],

       [[0.462]],

       [[0.35 ]],

       [[0.404]],

       [[0.364]],

       [[0.272]],

       [[0.392]],

       [[0.304]],

       [[0.29 ]],

       [[0.312]],

       [[0.278]],

       [[0.332]],

       [[0.296]],

       [[0.368]],

       [[0.42 ]],

       [[0.254]],

       [[0.29 ]],

       [[0.264]],

       [[0.262]],

       [[0.27 ]],

       [[0.378]],

       [[0.4  ]],

       [[0.42 ]],

       [[0.494]],

       [[0.616]],

       [[0.698]],

       [[0.532]],

       [[0.506]],

       [[0.494]],

       [[0.424]],

       [[0.386]],

       [[0.4  ]],

       [[0.332]],

       [[0.288]],

       [[0.388]],

       [[0.394]],

       [[0.41 ]],

       [[0.5  ]],

       [[0.4

In [0]:
class FCLayer:
    def __init__(self, input_size, output_size):
        self.input_size = input_size
        self.output_size = output_size
        self.weights = np.random.randn(input_size, output_size) / np.sqrt(input_size + output_size)
        self.bias = np.random.randn(1, output_size) / np.sqrt(input_size + output_size)

    def forward(self, input):
        self.input = input
        return np.dot(input, self.weights) + self.bias

    def backward(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        # bias_error = output_error
        
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

In [0]:
class ActivationLayer:
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime
    
    def forward(self, input):
        self.input = input
        return self.activation(input)
    
    def backward(self, output_error, learning_rate):
        return output_error * self.activation_prime(self.input)

In [0]:
# bonus
class SoftmaxLayer:
    def __init__(self, input_size):
        self.input_size = input_size
    
    def forward(self, input):
        self.input = input
        tmp = np.exp(input)
        self.output = tmp / np.sum(tmp)
        return self.output
    
    def backward(self, output_error, learning_rate):
        input_error = np.zeros(output_error.shape)
        out = np.tile(self.output.T, self.input_size)
        return self.output * np.dot(output_error, np.identity(self.input_size) - out)

In [0]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_prime(x):
    return np.exp(-x) / (1 + np.exp(-x))**2

def tanh(x):
    return np.tanh(x)

def tanh_prime(x):
    return 1 - np.tanh(x)**2

def relu(x):
    return np.maximum(x, 0)

def relu_prime(x):
    return np.array(x >= 0).astype('int')

In [0]:
def mse(y_true, y_pred):
    return np.mean(np.power(y_true - y_pred, 2))

def mse_prime(y_true, y_pred):
    return 2 * (y_pred - y_true) / y_pred.size

def sse(y_true, y_pred):
    return 0.5 * np.sum(np.power(y_true - y_pred, 2))

def sse_prime(y_true, y_pred):
    return y_pred - y_true

In [0]:
network = [
    FCLayer(13, 26),
    ActivationLayer(sigmoid, sigmoid_prime),
    FCLayer(26, 1)
]
epochs = 10000
learning_rate = 0.001
# training
for epoch in range(epochs):
    error = 0
    for xi, y_true in zip(y, x):
        # forward
        output = xi
        for layer in network:
            output = layer.forward(output)
        
        # error (display purpose only)
        error += sse(y_true, output)

        # backward
        output_error = sse_prime(y_true, output)
        for layer in reversed(network):
            output_error = layer.backward(output_error, learning_rate)
    
    error /= len(y)
    print('%d/%d, error=%f' % (epoch + 1, epochs, error))

Streaming output truncated to the last 5000 lines.
5002/10000, error=0.004025
5003/10000, error=0.004013
5004/10000, error=0.004003
5005/10000, error=0.004065
5006/10000, error=0.004043
5007/10000, error=0.004058
5008/10000, error=0.004080
5009/10000, error=0.004060
5010/10000, error=0.004017
5011/10000, error=0.004065
5012/10000, error=0.004044
5013/10000, error=0.004059
5014/10000, error=0.004024
5015/10000, error=0.004026
5016/10000, error=0.004024
5017/10000, error=0.004044
5018/10000, error=0.004038
5019/10000, error=0.004015
5020/10000, error=0.004071
5021/10000, error=0.004034
5022/10000, error=0.004044
5023/10000, error=0.004015
5024/10000, error=0.004054
5025/10000, error=0.004036
5026/10000, error=0.004027
5027/10000, error=0.004041
5028/10000, error=0.004037
5029/10000, error=0.004026
5030/10000, error=0.004014
5031/10000, error=0.004061
5032/10000, error=0.004071
5033/10000, error=0.004054
5034/10000, error=0.004026
5035/10000, error=0.004098
5036/10000, error=0.004055
5037

In [0]:
def predict(network, input):
    output = input
    for layer in network:
        output = layer.forward(output)
    return output

In [0]:
ans  = predict(network, [[0.16902  , 0.00 , 25.650 , 0 , 0.5810 , 5.9860 , 88.40 , 1.9929 ,  2 , 188.0 , 19.10 ,385.02 , 14.81]])

In [0]:
print(ans)

[[0.41808692]]


In [0]:
print(ans * 50)

array([[20.90434604]])